# **Для работы с Selenium**

In [6]:
%pip install -q google-colab-selenium

In [7]:
import google_colab_selenium as gs
from selenium.webdriver.chrome.options import Options

options = Options()

options.add_argument("--window-size=1920,1080")  # устанавливаем размер окна
options.add_argument("--disable-infobars")  # отключаем информационную панель
options.add_argument("--disable-popup-blocking")  # отключаем всплывающие окна
options.add_argument("--ignore-certificate-errors")  # игнорируем ошибки сертификатов
options.add_argument("--incognito")  # используем браузер в режиме инкогнито


driver = gs.Chrome(options=options)

driver.get('https://rivegauche.ru/brands')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Настраиваем логирование**

In [8]:
import logging

logging.basicConfig(
    filename='rive_gauche_parse.log',
    level=logging.INFO,
    format='scraping | %(asctime)s | %(levelname)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    force=True
)
logging.info("Тестовая запись в лог")

# **Извлечение брендов**

In [9]:
from selenium.webdriver.common.by import By

brands = driver.find_elements(By.XPATH, '//a[contains(@href, "/brands/")]') # ищем ссылки, содержащие "/brands/"

In [10]:
brand_links = []

In [11]:
for brand in brands:
    href = brand.get_attribute('href') # берем ссылку на бренд
    name = brand.get_attribute('text') # берем название бренда
    if href and name:
        brand_links.append([name, href])

In [12]:
brand_links

[[' ACCA KAPPA ', 'https://rivegauche.ru/brands/acca-kappa'],
 [' ACCOJE ', 'https://rivegauche.ru/brands/accoje'],
 [' ACQUA DI PARMA ', 'https://rivegauche.ru/brands/acqua-di-parma'],
 [' ACQUE DI ITALIA ', 'https://rivegauche.ru/brands/acque-di-italia'],
 [' A-DERMA ', 'https://rivegauche.ru/brands/a-derma'],
 [' ADOPT’ ', 'https://rivegauche.ru/brands/adopt'],
 [' AESOP ', 'https://rivegauche.ru/brands/aesop'],
 [' AHA ', 'https://rivegauche.ru/brands/aha'],
 [' AHAVA ', 'https://rivegauche.ru/brands/ahava'],
 [' AJMAL ', 'https://rivegauche.ru/brands/ajmal'],
 [' AKRO ', 'https://rivegauche.ru/brands/akro'],
 [' ALAN BRAY ', 'https://rivegauche.ru/brands/alan-bray'],
 [' ALAN HADASH ', 'https://rivegauche.ru/brands/alan-hadash'],
 [' ALERANA ru', 'https://rivegauche.ru/brands/alerana'],
 [' ALEXANDRE.J ', 'https://rivegauche.ru/brands/alexandre-j'],
 [' ALFAPARF MILANO ', 'https://rivegauche.ru/brands/alfaparf-milano'],
 [' ALGOLOGIE ', 'https://rivegauche.ru/brands/algologie'],
 

# **Получение ссылок на страницы продуктов**

In [13]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [14]:
products_data = []

In [ ]:
driver = gs.Chrome(options=options)

for brand in brand_links:
    brand_name, brand_link = brand
    driver.get(brand_link)

    # Ищем ссылки, содержащие "/product/"
    try:
        WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.XPATH, '//a[contains(@href, "/product/")]'))
        )
    # если ссылка не найдена в течение 1 секунды
    except:
        logging.info(f"Продукты не найдены для {brand_name}. Продолжаем...")
        continue

    # чтобы избежать дубликатов используем множество
    product_links_set = set()

    # скролл страницы, пока товары появляются
    last_count = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # скролл
        time.sleep(1)

        product_links = driver.find_elements(By.XPATH, '//a[contains(@href, "/product/")]')

        for product in product_links:
            product_href = product.get_attribute("href")
            if product_href:
                product_links_set.add(product_href)

        if len(product_links_set) == last_count:
            break
        last_count = len(product_links_set)

    for product_href in product_links_set:
        products_data.append([brand_name, product_href])

    logging.info(f"Ссылки на продукты добавлены для {brand_name}")

driver.quit()

In [16]:
products_data

[[' ACCA KAPPA ',
  'https://rivegauche.ru/product/acca-kappa-muschio-bianco-white-moss-soothing-hand-wash'],
 [' ACCA KAPPA ',
  'https://rivegauche.ru/product/acca-kappa-natural-care-bianco-perla-black-toothpaste'],
 [' ACCA KAPPA ',
  'https://rivegauche.ru/product/acca-kappa-muschio-bianco-white-moss-deodorant-stick'],
 [' ACCA KAPPA ',
  'https://rivegauche.ru/product/acca-kappa-1869-purifying-protective-deodorant-spray'],
 [' ACCA KAPPA ',
  'https://rivegauche.ru/product/acca-kappa-muschio-bianco-eau-de-parfum-15'],
 [' ACCA KAPPA ',
  'https://rivegauche.ru/product/acca-kappa-muschio-bianco-white-moss-shaving-foam'],
 [' ACCA KAPPA ',
  'https://rivegauche.ru/product/acca-kappa-muschio-bianco-white-moss-deodorant-spray'],
 [' ACCA KAPPA ',
  'https://rivegauche.ru/product/acca-kappa-volata-eau-de-parfum'],
 [' ACCA KAPPA ',
  'https://rivegauche.ru/product/acca-kappa-vintage-collection-soft-neylon-toothbrush-ivory-white'],
 [' ACCA KAPPA ',
  'https://rivegauche.ru/product/acca

# Теперь для каждого продукта читаем информацию

In [17]:
import requests
import json
from bs4 import BeautifulSoup
import re
from requests.exceptions import RequestException

In [18]:
detailed_products_data = []

In [19]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [20]:
def process_product_page(product_href):
    try:
        response = requests.get(product_href, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        description_meta = soup.find('meta', {'itemprop': 'description'})
        description = description_meta.get('content') if description_meta else None
        table = soup.find('table')
        if table:
            rows = table.find_all('tr')
        else:
            rows = []
        size = None
        category = None
        production = None
        result = None

        for row in rows:
            cells = row.find_all('td')
            if len(cells) >= 2:
                label = cells[0].get_text(strip=True)
                value = cells[1].get_text(strip=True)

                if label in ['Объем, мл', 'Вес, г']:
                    size = f"{value} {'мл' if label == 'Объем, мл' else 'г'}"
                elif label == 'Категория':
                    category = value
                elif label == 'Производство':
                    production = value
                elif label == 'Результат':
                    result = value

        return description, size, category, production, result

    except Exception as e:
        logging.warning(f"Ошибка при обработке страницы продукта {product_href}: {str(e)}")
        return None, None, None, None, None

In [ ]:
driver = gs.Chrome(options=options)

for product in products_data:
    brand_name, product_href = product
    driver.get(product_href)

    # Ждем загрузки товара
    try:
        WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.TAG_NAME, "h1"))
        )
    except:
        logging.info(f"Товар не загрузился: {product_href}. Продолжаем...")
        continue

    # 1. название товара
    try:
        product_name = driver.find_element(By.TAG_NAME, "h1").text.strip()
    except:
        product_name = None

    # 2. рейтинг
    try:
        rating = driver.find_element(By.XPATH, '//div[@class="rating"]/span').text.strip()
    except:
        rating = None

    # 3. количество отзывов
    try:
        reviews_count = driver.find_element(By.XPATH, '//div[@class="reviews-count"]').text.strip()
        reviews_count = int(re.sub(r'\D', '', reviews_count))
    except:
        reviews_count = 0

    # 4. хит/не хит
    try:
        is_hit = True if driver.find_element(By.XPATH, '//div[contains(text(), " Хит ")]') else False
    except:
        is_hit = False

    # 5. только в магазинах
    try:
        only_in_stores = True if driver.find_element(By.XPATH, '//div[contains(text(), " Только для самовывоза из магазина ")]') else False
    except:
        only_in_stores = False

    # 6. цена
    try:
        price = driver.find_element(By.XPATH, '//div/span[@class="base-value"]').text.strip()
        price = int(re.sub(r'\D', '', price))
    except:
        price = None

    # 7. описание, размер, категория, производство, результат
    description, size, category, production, result = process_product_page(product_href)


    detailed_products_data.append([
        brand_name, product_href, product_name, rating, reviews_count, price, is_hit, only_in_stores, description, size, category, production, result
    ])

    logging.info(f"Добавлен продукт {product_name}")

driver.quit()

In [ ]:
detailed_products_data

[['Acca Kappa',
  'https://rivegauche.ru/product/acca-kappa-volata-eau-de-parfum',
  'Acca Kappa Volata Eau de Parfum',
  None,
  0,
  2990,
  False,
  False,
  'Volata - официальный аромат велогонки Giro d’Italia.<br><br>Свежие нотки бергамота сочетаются со слегка пряными нотами розового перца и имбиря.<br><br>Невероятно элегантный ирис, харизматичное сердце этого аромата, сочетается с согревающими душу нотами лаванды и ангелики.<br><br>Древесный аромат кипариса в качестве базовой ноты придает нотку стиля непотворимому сочетанию.<br><br>Квинтэссенция ароматов и оттенков этой страны.<br><br>Верхние ноты: Ббергамот, розовый перец, имбирь и кардамон<br>Ноты сердца: ирис, бобы тонка, лаванда, лабданум и ангелика<br>Базовые ноты: пачули, ветивер, кипарис, ваниль и мускус',
  '15 мл',
  'Парфюмерия',
  'Италия',
  None],
 ['Acca Kappa',
  'https://rivegauche.ru/product/acca-kappa-natural-care-natural-toothpaste',
  'Acca Kappa Natural Care Natural Toothpaste',
  '5',
  1,
  1850,
  False,
 

In [ ]:
df_products_data = pd.DataFrame(detailed_products_data,
                                columns=['brand_name', 'product_link', 'product_name',
                                         'rating', 'reviews_count', 'price', 'is_hit',
                                         'only_in_stores', 'description', 'size', 'category',
                                         'production', 'result'])

In [ ]:
df_products_data.to_csv('rive_gauche_products.csv', encoding='utf-8-sig', index=False)